#Run these first

In [ ]:
%pip install matplotlib==3.6.2

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from scipy.misc import electrocardiogram
import pywt
import scipy.signal as signal
import seaborn as sns

from ipywidgets import interactive

In [ ]:
#import files here
norm_data = pd.read_csv("normal.csv")
hem_data = pd.read_csv("hemiplegia.csv")

In [ ]:
def get_values(data,key):
    
    wavelet_type='db6'

    DWTcoeffs = pywt.wavedec(data[key],wavelet_type,mode='symmetric', level=9, axis=-1)

    for i in range(1,10):
        DWTcoeffs[-i] = np.zeros_like(DWTcoeffs[-i])

    filtered_data_dwt=pywt.waverec(DWTcoeffs,wavelet_type,mode='symmetric',axis=-1) 

    fc = 0.9  # Cut-off frequency of the butterworth filter
    w = fc / (100 / 2) # Normalize the frequency
    b, a = signal.butter(5, w, 'low')
    filtered_data_butterworth = signal.filtfilt(b, a, data[key])

    return filtered_data_dwt, filtered_data_butterworth


#Testing

In [ ]:
start1 = 65000
duration1 = 600

def create_graph1(start=65000,duration=600):
    start1 , duration1 = start , duration
    data1 = norm_data[start:start+duration] 
    
    filtered_data_dwt, filtered_data_butterworth = get_values(data1,'Ay')

    f1 = plt.figure(1,figsize=(6.85,4))
    plt.psd(data1['Ay'], NFFT=512, Fs=360, color='blue', label='original data')
    plt.psd(filtered_data_dwt, NFFT=512, Fs=360, color='red', label='filtered data (DWT)')
    plt.psd(filtered_data_butterworth, NFFT=512, Fs=360, color='black', label='filtered data (Butterworth)')
    plt.legend()

    f2 = plt.figure(2,figsize=(6.85,5))
    plt.subplot(311)
    plt.plot(data1['Ay'], color='blue', label='original data')
    plt.title('original')
    plt.legend() 
    plt.grid()
    plt.subplot(312)
    plt.plot(filtered_data_dwt, color='red', label='filtered data (DWT)')
    plt.title('filtered (DWT)')
    plt.grid()
    plt.legend()
    plt.subplot(313)
    plt.plot(filtered_data_butterworth, color='black', label='filtered data (Butterworth)')
    plt.title('filtered (Butterworth)')
    plt.grid()
    plt.legend()
    # f2.supxlabel('time in milliseconds →')
    # f2.supylabel('Ay (acceleration in y direction) →')
    plt.tight_layout()
w1 = interactive(create_graph1,start=(0,80000),duration=(500,1500))
w1

In [ ]:
start2 = 20000
duration2 = 600

def create_graph2(start=20000,duration=600):
    start2 , duration2 = start , duration
    data2 = hem_data[start:start+duration] 
    
    filtered_data_dwt, filtered_data_butterworth = get_values(data2,'Ay')

    f1 = plt.figure(1,figsize=(6.85,4))
    plt.psd(data2['Ay'], NFFT=512, Fs=360, color='blue', label='original data')
    plt.psd(filtered_data_dwt, NFFT=512, Fs=360, color='red', label='filtered data (DWT)')
    plt.psd(filtered_data_butterworth, NFFT=512, Fs=360, color='black', label='filtered data (Butterworth)')
    plt.legend()

    f2 = plt.figure(2,figsize=(6.85,5))
    plt.subplot(311)
    plt.plot(data2['Ay'], color='blue', label='original data')
    plt.title('original')
    plt.grid()
    plt.legend()
    plt.subplot(312)
    plt.plot(filtered_data_dwt, color='red', label='filtered data (DWT)')
    plt.title('filtered (DWT)')
    plt.grid()
    plt.legend()
    plt.subplot(313)
    plt.plot(filtered_data_butterworth, color='black', label='filtered data (Butterworth)')
    plt.title('filtered (Butterworth)')
    plt.grid()
    plt.legend()
    # f2.supxlabel('time in milliseconds →')
    # f2.supylabel('Ay (acceleration in y direction) →')
    plt.tight_layout()
w2 = interactive(create_graph2,start=(0,27000),duration=(500,1500))
w2

#Graph1 (long)

In [ ]:
# Normal & Hemiplegia 
norm_data_slice = norm_data[start1:start1+duration1]  
hem_data_slice = hem_data[start2:start2+duration2]  
val = ['Ax','Gx','Ay','Gy','Az','Gz']
f1 = plt.figure(constrained_layout=True,figsize=(14, 4.5))
ax1 = f1.subfigures(3,2)
row , col = 3 , 2
for i in range(row): 
    for j in range(col):
        ax2 = ax1[i,j].subplots(1,2,sharex=True,sharey=True,squeeze=False)
        l1=ax2[0,0].plot(norm_data_slice.set_index(pd.Index(range(duration1)))[val[i*col+j]], color='C2', label='Normal',linewidth=0.9)
        ax2[0,0].set_title(val[i*col+j]+' N')
        ax2[0,0].grid()
        l2=ax2[0,1].plot(hem_data_slice.set_index(pd.Index(range(duration2)))[val[i*col+j]], color='C3', label='Hemiplegia',linewidth=0.9)
        ax2[0,1].set_title(val[i*col+j]+' H')
        ax2[0,1].grid()
f1.suptitle("Normal (N) & Hemiplegia (H)")
f1.text(0.5, -0.02, 'time →', ha='center')
f1.text(-0.03, 0.5, 'Magnitude →', va='center', rotation='vertical')
# Printing legend
f1.legend(['Normal','Hemiplegia'],loc='center',ncol=2,bbox_to_anchor=(0.5, -0.075),edgecolor='0.4')
# f1.tight_layout()

#Graph 1 (short)

In [ ]:
# Normal & Hemiplegia 
norm_data_slice = norm_data[start1:start1+duration1]  
hem_data_slice = hem_data[start2:start2+duration2]  
val = ['Ax','Gx','Ay','Gy','Az','Gz']
f1 = plt.figure(constrained_layout=True,figsize=(6, 4.5))
ax1 = f1.subfigures(3,2)
row , col = 3 , 2
for i in range(row): 
    for j in range(col):
        ax2 = ax1[i,j].subplots(1,2,sharex=True,sharey=True,squeeze=False)
        l1=ax2[0,0].plot(norm_data_slice.set_index(pd.Index(range(duration1)))[val[i*col+j]], color='C2', label='Normal',linewidth=0.9)
        ax2[0,0].set_title(val[i*col+j]+' N')
        ax2[0,0].grid()
        l2=ax2[0,1].plot(hem_data_slice.set_index(pd.Index(range(duration2)))[val[i*col+j]], color='C3', label='Hemiplegia',linewidth=0.9)
        ax2[0,1].set_title(val[i*col+j]+' H')
        ax2[0,1].grid()
f1.suptitle("Normal (N) & Hemiplegia (H)")
f1.text(0.5, -0.02, 'time →', ha='center')
f1.text(-0.03, 0.5, 'Magnitude →', va='center', rotation='vertical')
# Printing legend
f1.legend(['Normal','Hemiplegia'],loc='center',ncol=2,bbox_to_anchor=(0.5, -0.075),edgecolor='0.4')
# f1.tight_layout()

#Graph 2

In [ ]:
norm_data_slice = norm_data[start1:start1+duration1] 
hem_data_slice = hem_data[start2:start2+duration2] 
val = ['Ax','Ay','Az','Gx','Gy','Gz']
f1, ax1 = plt.subplots(2,3,figsize=(8, 5),sharex=True,sharey=True)

row , col = 2 , 3
for i in range(row): 
    for j in range(col):
        filtered_data_dwt_norm, filtered_data_butterworth_norm = get_values(norm_data_slice,val[i*col+j])
        filtered_data_dwt_hem, filtered_data_butterworth_hem = get_values(hem_data_slice,val[i*col+j])
        # normal
        ax1[i,j].psd(norm_data_slice[val[i*col+j]], NFFT=512, Fs=100, color='C0', label='Raw Normal',linewidth=0.95)
        ax1[i,j].psd(filtered_data_dwt_norm, NFFT=512, Fs=100, color='C2', label='Inverse wavelet transformed Normal',ls='dashed')
        ax1[i,j].psd(filtered_data_butterworth_norm, NFFT=512, Fs=100, color='C9', label='Butterworth normal',ls='-.',linewidth=1.5)
        # hemiplegia
        ax1[i,j].psd(hem_data_slice[val[i*col+j]], NFFT=512, Fs=100, color='C1', label='Raw Hemiplegia',linewidth=0.95)
        ax1[i,j].psd(filtered_data_dwt_hem, NFFT=512, Fs=100, color='C3', label='Inverse wavelet transformed Hemiplegia',ls='dashed')
        ax1[i,j].psd(filtered_data_butterworth_hem, NFFT=512, Fs=100, color='C6', label='Butterworth Hemiplegia',ls='-.',linewidth=1.5)
        ax1[i,j].set_title(val[i*col+j])
        ax1[i,j].set(xlabel=None,ylabel=None)
        ax1[i,j].tick_params(axis="y", labelsize=8)
f1.text(0.5, 0, 'Frequency (Hz) →', ha='center')
f1.text(0, 0.5, 'Power Spectral Density (dB/Hz) →', va='center', rotation='vertical')
# Printing legend
handles, labels = f1.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
f1.legend(by_label.values(), by_label.keys(),loc='center',ncol=2,bbox_to_anchor=(0.5, -0.1),edgecolor='0.4')
f1.tight_layout()
plt.savefig('Graph2.jpg', dpi=300, bbox_inches='tight')

#Graph 3 (Version 1)

In [ ]:
# Normal 
norm_data_slice = norm_data[start1:start1+duration1]  
val = ['Ax','Gx','Ay','Gy','Az','Gz']
f1, ax1 = plt.subplots(3,2,figsize=(6, 5),sharex=False,sharey=False)
row , col = 3 , 2
for i in range(row): 
    for j in range(col):
        filtered_data_dwt_norm, filtered_data_butterworth_norm = get_values(norm_data_slice,val[i*col+j])
        ax1[i,j].plot(norm_data_slice.set_index(pd.Index(range(duration1)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax1[i,j].plot(filtered_data_dwt_norm, color='C2', label='Wavelet',ls='dashed')
        ax1[i,j].plot(filtered_data_butterworth_norm, color='C3', label='Butterworth',ls='-.')
        ax1[i,j].set_title(val[i*col+j])
        ax1[i,j].grid()
f1.suptitle("Normal")
f1.text(0.5, -0.02, 'time →', ha='center')
f1.text(0, 0.5, 'Magnitude →', va='center', rotation='vertical')
# Printing legend
handles, labels = f1.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
f1.legend(by_label.values(), by_label.keys(),loc='center',ncol=3,bbox_to_anchor=(0.5, -0.1),edgecolor='0.4')
f1.tight_layout()

In [ ]:
# Hemiplegia 
hem_data_slice = hem_data[start2:start2+duration2]  
val = ['Ax','Gx','Ay','Gy','Az','Gz']
f1, ax1 = plt.subplots(3,2,figsize=(6, 5),sharex=False,sharey=False)
row , col = 3 , 2
for i in range(row): 
    for j in range(col):
        filtered_data_dwt_hem, filtered_data_butterworth_hem = get_values(hem_data_slice,val[i*col+j])
        ax1[i,j].plot(hem_data_slice.set_index(pd.Index(range(duration2)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax1[i,j].plot(filtered_data_dwt_hem, color='C2', label='Wavelet',ls='dashed')
        ax1[i,j].plot(filtered_data_butterworth_hem, color='C3', label='Butterworth',ls='-.')
        ax1[i,j].set_title(val[i*col+j])
        ax1[i,j].grid()
f1.suptitle("Hemiplegia")
f1.text(0.5, -0.02, 'time →', ha='center')
f1.text(0, 0.5, 'Magnitude →', va='center', rotation='vertical')
# Printing legend
handles, labels = f1.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
f1.legend(by_label.values(), by_label.keys(),loc='center',ncol=3,bbox_to_anchor=(0.5, -0.1),edgecolor='0.4')
f1.tight_layout()

#Graph 3 (Version 2)

In [ ]:
# Normal & Hemiplegia 
norm_data_slice = norm_data[start1:start1+duration1]  
hem_data_slice = hem_data[start2:start2+duration2]  
val = ['Ax','Gx','Ay','Gy','Az','Gz']
f1 = plt.figure(constrained_layout=True,figsize=(6, 8))
ax1 = f1.subfigures(3,2)
row , col = 3 , 2
for i in range(row): 
    for j in range(col):
        ax2 = ax1[i,j].subplots(2,1,sharex=True,sharey=False,squeeze=False)
        filtered_data_dwt_norm, filtered_data_butterworth_norm = get_values(norm_data_slice,val[i*col+j])
        ax2[0,0].plot(norm_data_slice.set_index(pd.Index(range(duration1)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax2[0,0].plot(filtered_data_dwt_norm, color='C2', label='Wavelet',ls='dashed')
        ax2[0,0].plot(filtered_data_butterworth_norm, color='C3', label='Butterworth',ls='-.')
        ax2[0,0].set_title(val[i*col+j]+' (Up: N Down: H)')
        ax2[0,0].grid()
        filtered_data_dwt_hem, filtered_data_butterworth_hem = get_values(hem_data_slice,val[i*col+j])
        ax2[1,0].plot(hem_data_slice.set_index(pd.Index(range(duration2)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax2[1,0].plot(filtered_data_dwt_hem, color='C2', label='Wavelet',ls='dashed')
        ax2[1,0].plot(filtered_data_butterworth_hem, color='C3', label='Butterworth',ls='-.')
        ax2[1,0].grid()
f1.suptitle("Normal (N) & Hemiplegia (H)")
f1.text(0.5, -0.02, 'time →', ha='center')
f1.text(-0.03, 0.5, 'Magnitude →', va='center', rotation='vertical')
# Printing legend
handles, labels = f1.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
f1.legend(by_label.values(), by_label.keys(),loc='center',ncol=3,bbox_to_anchor=(0.5, -0.05),edgecolor='0.4')
# f1.tight_layout()

#Graph 3 (Version 3)

In [ ]:
# Normal & Hemiplegia 
norm_data_slice = norm_data[start1:start1+duration1]  
hem_data_slice = hem_data[start2:start2+duration2]  
val = ['Ax','Gx','Ay','Gy','Az','Gz']
f1 = plt.figure(constrained_layout=True,figsize=(8, 4.5))
ax1 = f1.subfigures(3,2)
row , col = 3 , 2
for i in range(row): 
    for j in range(col):
        ax2 = ax1[i,j].subplots(1,2,sharex=True,sharey=True,squeeze=False)
        filtered_data_dwt_norm, filtered_data_butterworth_norm = get_values(norm_data_slice,val[i*col+j])
        ax2[0,0].plot(norm_data_slice.set_index(pd.Index(range(duration1)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax2[0,0].plot(filtered_data_dwt_norm, color='C2', label='Wavelet',ls='dashed')
        ax2[0,0].plot(filtered_data_butterworth_norm, color='C3', label='Butterworth',ls='-.')
        ax2[0,0].set_title(val[i*col+j]+' N')
        ax2[0,0].grid()
        filtered_data_dwt_hem, filtered_data_butterworth_hem = get_values(hem_data_slice,val[i*col+j])
        ax2[0,1].plot(hem_data_slice.set_index(pd.Index(range(duration2)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax2[0,1].plot(filtered_data_dwt_hem, color='C2', label='Wavelet',ls='dashed')
        ax2[0,1].plot(filtered_data_butterworth_hem, color='C3', label='Butterworth',ls='-.')
        ax2[0,1].set_title(val[i*col+j]+' H')
        ax2[0,1].grid()
f1.suptitle("Normal (N) & Hemiplegia (H)")
f1.text(0.5, -0.02, 'time →', ha='center')
f1.text(-0.03, 0.5, 'Magnitude →', va='center', rotation='vertical')
# Printing legend
handles, labels = f1.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
f1.legend(by_label.values(), by_label.keys(),loc='center',ncol=3,bbox_to_anchor=(0.5, -0.075),edgecolor='0.4')
# f1.tight_layout()

#Graph3 (Version 4)

In [ ]:
# Normal & Hemiplegia 
norm_data_slice = norm_data[start1:start1+duration1]  
hem_data_slice = hem_data[start2:start2+duration2]  
val = ['Ax','Ay','Az','Gx','Gy','Gz']
f1 = plt.figure(constrained_layout=True,figsize=(14, 4.5))
ax1 = f1.subfigures(2,3)
row , col = 2 , 3
for i in range(row): 
    for j in range(col):
        ax2 = ax1[i,j].subplots(1,2,sharex=True,sharey=True,squeeze=False)
        filtered_data_dwt_norm, filtered_data_butterworth_norm = get_values(norm_data_slice,val[i*col+j])
        ax2[0,0].plot(norm_data_slice.set_index(pd.Index(range(duration1)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax2[0,0].plot(filtered_data_dwt_norm, color='C2', label='Wavelet',ls='dashed')
        ax2[0,0].plot(filtered_data_butterworth_norm, color='C3', label='Butterworth',ls='-.')
        ax2[0,0].set_title(val[i*col+j]+' N')
        ax2[0,0].grid()
        filtered_data_dwt_hem, filtered_data_butterworth_hem = get_values(hem_data_slice,val[i*col+j])
        ax2[0,1].plot(hem_data_slice.set_index(pd.Index(range(duration2)))[val[i*col+j]], color='C0', label='Raw',linewidth=0.9)
        ax2[0,1].plot(filtered_data_dwt_hem, color='C2', label='Wavelet',ls='dashed')
        ax2[0,1].plot(filtered_data_butterworth_hem, color='C3', label='Butterworth',ls='-.')
        ax2[0,1].set_title(val[i*col+j]+' H')
        ax2[0,1].grid()
f1.suptitle("Normal (N) & Hemiplegia (H)")
f1.text(0.5, -0.02, 'time →', ha='center')
f1.text(-0.03, 0.5, 'Magnitude →', va='center', rotation='vertical')
# Printing legend
handles, labels = f1.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
f1.legend(by_label.values(), by_label.keys(),loc='center',ncol=3,bbox_to_anchor=(0.5, -0.075),edgecolor='0.4')
# f1.tight_layout()

# Graph 4

In [ ]:
norm_data_slice = norm_data[start1:start1+duration1].copy()
hem_data_slice = hem_data[start2:start2+duration2].copy()

scaler = MinMaxScaler()
fit = scaler.fit(norm_data_slice[['Ax','Ay','Az','Gx','Gy','Gz']])
norm_data_slice[['Ax','Ay','Az','Gx','Gy','Gz']] = fit.transform(norm_data_slice[['Ax','Ay','Az','Gx','Gy','Gz']])
hem_data_slice[['Ax','Ay','Az','Gx','Gy','Gz']] = fit.transform(hem_data_slice[['Ax','Ay','Az','Gx','Gy','Gz']])

# norm_data_slice
f1,ax1 = plt.subplots(1,1,figsize=(5,3.5),sharey=True)
norm_data_slice_g = norm_data_slice.melt(value_vars=['Ax','Ay','Az','Gx','Gy','Gz'])
hem_data_slice_g = hem_data_slice.melt(value_vars=['Ax','Ay','Az','Gx','Gy','Gz'])
sns.swarmplot(data=hem_data_slice_g,x="variable",y="value",size=1.25, ax=ax1,zorder=0)
sns.boxplot(data=norm_data_slice_g,x="variable",y="value",boxprops={'facecolor':'None'},showfliers=False,showmeans=True,meanprops={"marker":"o","mfc":"orange","mec":'y'}, ax=ax1).set(title='',xlabel=None,ylabel=None)
f1.text(0.5, 0, 'Features →', ha='center',size=12,fontweight='bold')
f1.text(0, 0.5, 'Scaled Magnitude →', va='center', rotation='vertical',size=12,fontweight='bold')
f1.suptitle("Hemiplegia (Swarmplot) vs Normal (Boxplot)")
ax1.xaxis.grid(True) # Show the vertical gridlines
ax1.yaxis.grid(True) 
f1.tight_layout()
plt.savefig('Graph4.jpg', dpi=300)